<a href="https://colab.research.google.com/github/Wilson30678tw/IAT460_A2/blob/main/Lin_Che_wei_rulebased_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# First, let's import our required libraries
from collections import defaultdict
import random

In [35]:
# 定義故事語法
story_grammar = {
'STORY': [[
    "Long ago, {CHARACTER} roamed {PLACE}, searching for purpose. One day, they {ACTION} and stumbled upon {PROBLEM}, unknowingly setting forth an irreversible fate.",
    "As they ventured deeper, {ENEMY} emerged from the shadows, driven by {MOTIVATION}. Despite the peril, {CHARACTER} pressed forward, determined to fulfill their destiny.",
    "The air crackled with magic as steel clashed against dark sorcery. The battle raged on, and the land trembled under their struggle. Just as defeat seemed imminent, a beacon of hope emerged—{ALLY} arrived and revealed a hidden truth.",
    "Summoning their last reserves of strength, {CHARACTER} and {ALLY} combined forces. With a final, desperate attempt, {ABILITY} was unleashed, striking at the heart of darkness.",
    "As dawn broke over {PLACE}, silence fell upon the battlefield. {CHARACTER}, forever changed, gazed upon the world they had saved. Though victories are fleeting, legends endure. The darkness was defeated, and {OUTCOME}."
]],


    # Characters & Places
    'CHARACTER': ['a wandering knight', 'The Lost Wizard', 'an elf archer', 'a group of rogue thief'],
    'PLACE': ['in the ancient ruins', 'within the cursed forest', 'beyond the forgotten kingdom', 'under the dragon’s lair'],

    # Actions & Problems
    'ACTION': ['discovered', 'was searching for', 'was ambushed in', 'encountered'],
    'PROBLEM': ['a sealed portal', 'a forbidden spellbook', 'a hidden artifact', 'A prophecy of doom and gloom'],

    # Enemy & Motivation
    'ENEMY': ['a dark sorcerer', 'a wicked warlord', 'a cursed dragon', 'an ancient demon'],
    'MOTIVATION': ['seeks to conquer the kingdom', 'plans to steal the artifact', 'desires to break an ancient curse', 'aims to overthrow the gods'],

    # Ally, Ability & Outcome
    'ALLY': ['a wise old mage', 'a cunning rogue', 'a celestial spirit', 'a talking owl'],
    'ABILITY': ['used a powerful spell', 'unleashed elemental magic', 'discovered a hidden passage', 'activated an ancient relic'],
    'OUTCOME': ['and banished the darkness', 'and sealed the enemy away forever', 'and restored balance to the world', 'and reclaimed the lost treasure']
}

# CFG 生成函數
def generate_text(symbol, grammar):
    if isinstance(symbol, str) and symbol in grammar:
        production = random.choice(grammar[symbol])
        if isinstance(production, list):
            return ' '.join(generate_text(sym, grammar) for sym in production)
        return production
    return symbol

def generate_story():
     return ' '.join(generate_text('STORY', story_grammar))



Markov Model in here

In [36]:
def build_markov_chain(text, order=4):
    chain = defaultdict(list)
    words = text.split()

    for i in range(len(words) - order):
        state = tuple(words[i:i+order])
        next_word = words[i+order]
        chain[state].append(next_word)

    return chain

def generate_markov_text(chain, num_words=3):
    if not chain:
        return ""
    words = list(random.choice(list(chain.keys())))
    order = len(list(chain.keys())[0])

    for _ in range(num_words - order):
        state = tuple(words[-order:])
        if state in chain and chain[state]:
            next_word = random.choice(chain[state])
            words.append(next_word)
        else:
            break

    return ' '.join(words)



In [37]:
sample_text = """
In the heart of an ancient kingdom, a lost prince was drawn to a celestial tower that touches the clouds. Legends spoke of the tower as a place where the past and future converged, where whispers of fate guided those who sought answers. As he ascended the marble steps, a strange glow pulsed from the runes carved into the walls.

However, something was not right. The air crackled with magic, and shadows shifted unnaturally. He felt a presence, unseen but undeniably watching.

Determined to uncover the truth, the prince encountered a hidden trial of the ancients. The great stone doors behind him slammed shut, sealing his path backward. Before him, the chamber twisted, revealing an ever-shifting labyrinth. Voices echoed in forgotten tongues, warning him of the trials ahead.
"""
"""Long ago, Sir Alden roamed the Forgotten Kingdom, searching for purpose. One day, he uncovered an ancient tomb and stumbled upon a slumbering curse, unknowingly setting forth an irreversible fate.
As he ventured deeper, the Wraith King emerged from the shadows, driven by his hunger for a lost soul. Despite the peril, Alden pressed forward, determined to fulfill his destiny.
The air crackled with magic as steel clashed against dark sorcery. The battle raged on, and the land trembled under their struggle. Just as defeat seemed imminent, a beacon of hope emerged—the ghost of Queen Lysara arrived and revealed a hidden truth.
Summoning their last reserves of strength, Alden and Lysara combined forces. With a final, desperate attempt, the Blade of Revenants was unleashed, striking at the heart of darkness.
As dawn broke over the Forgotten Kingdom, silence fell upon the battlefield. Alden, forever changed, gazed upon the world he had saved. Though victories are fleeting, legends endure. The darkness was defeated, and the kingdom, once cursed, began to bloom once more.
"""
"""Long ago, Vaelin the Starborn roamed the Astral Wastes, searching for purpose. One day, he followed the whispers of a dying star and stumbled upon a collapsing rift in space, unknowingly setting forth an irreversible fate.
As he ventured deeper, the Celestial Devourer emerged from the shadows, driven by its eternal hunger to consume the fabric of reality. Despite the peril, Vaelin pressed forward, determined to fulfill his destiny.
The air crackled with magic as steel clashed against dark sorcery. The battle raged on, and the land trembled under their struggle. Just as defeat seemed imminent, a beacon of hope emerged—the lost deity, Seraphis, arrived and revealed a hidden truth.
Summoning their last reserves of strength, Vaelin and Seraphis combined forces. With a final, desperate attempt, the Sigil of Eternity was unleashed, striking at the heart of darkness.
As dawn broke over the Astral Wastes, silence fell upon the battlefield. Vaelin, forever changed, gazed upon the world he had saved. Though victories are fleeting, legends endure. The darkness was defeated, and the rift sealed, ensuring the universe would live another age.

"""
# 建立 Markov Chain
markov_chain = build_markov_chain(sample_text, order=3)

def get_fallback(value, category):
    """Ensure a valid replacement, prefer Markov output if available."""
    generated_text = value.strip()

    # Ensure meaningful Markov text (allow more variation)
    if not generated_text or len(generated_text.split()) < 2:
        return generate_text(category, story_grammar)

    return generated_text

def generate_story():
    story_template = ' '.join(story_grammar['STORY'][0])

    return story_template.format(
        CHARACTER=random.choice(story_grammar['CHARACTER']) + " " + generate_markov_text(markov_chain, num_words=2),
        PLACE=generate_markov_text(markov_chain, num_words=4) or random.choice(story_grammar['PLACE']),
        ACTION=generate_markov_text(markov_chain, num_words=2) or random.choice(story_grammar['ACTION']),
        ENEMY=random.choice(story_grammar['ENEMY']) + " " + generate_markov_text(markov_chain, num_words=2),
        MOTIVATION=generate_markov_text(markov_chain, num_words=5) or random.choice(story_grammar['MOTIVATION']),
        PROBLEM=generate_markov_text(markov_chain, num_words=4) or random.choice(story_grammar['PROBLEM']),
        ALLY=random.choice(story_grammar['ALLY']) + " " + generate_markov_text(markov_chain, num_words=2),
        ABILITY=generate_markov_text(markov_chain, num_words=4) or random.choice(story_grammar['ABILITY']),
        OUTCOME=generate_markov_text(markov_chain, num_words=5) or random.choice(story_grammar['OUTCOME'])
    )


In [34]:
print(generate_story())

Long ago, a rogue thief runes carved into roamed a presence, unseen but, searching for purpose. One day, they place where the and stumbled upon glow pulsed from the, unknowingly setting forth an irreversible fate. As they ventured deeper, a wicked warlord ancient kingdom, a emerged from the shadows, driven by felt a presence, unseen but. Despite the peril, a rogue thief runes carved into pressed forward, determined to fulfill their destiny. The air crackled with magic as steel clashed against dark sorcery. The battle raged on, and the land trembled under their struggle. Just as defeat seemed imminent, a beacon of hope emerged—a celestial spirit Legends spoke of arrived and revealed a hidden truth. Summoning their last reserves of strength, a rogue thief runes carved into and a celestial spirit Legends spoke of combined forces. With a final, desperate attempt, steps, a strange glow was unleashed, striking at the heart of darkness. As dawn broke over a presence, unseen but, silence fell 

In [38]:
story = generate_story()
print(f"Word Count: {len(story.split())}")
print(story)

Word Count: 207
Long ago, a group of rogue thief path backward. Before roamed Legends spoke of the, searching for purpose. One day, they The air crackled and stumbled upon magic, and shadows shifted, unknowingly setting forth an irreversible fate. As they ventured deeper, a cursed dragon him slammed shut, emerged from the shadows, driven by the prince encountered a hidden. Despite the peril, a group of rogue thief path backward. Before pressed forward, determined to fulfill their destiny. The air crackled with magic as steel clashed against dark sorcery. The battle raged on, and the land trembled under their struggle. Just as defeat seemed imminent, a beacon of hope emerged—a wise old mage encountered a hidden arrived and revealed a hidden truth. Summoning their last reserves of strength, a group of rogue thief path backward. Before and a wise old mage encountered a hidden combined forces. With a final, desperate attempt, of the trials ahead. was unleashed, striking at the heart of dar